In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import re
import jieba
import jieba.posseg as psg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [3]:
content=requests.get('https://www.jjwxc.net/fenzhan/yq/bgx.html')
soup=BeautifulSoup(content.content,'lxml')
soup=soup.find('div', {'id' : 'bgx'})
title_all=soup.find_all('li') 

In [4]:
results={'Name':[], 'URL':[]}

In [5]:
for title in title_all:
    if title.a:
        t=title.a
        len_t=len(t.find_next_siblings('a'))
        if len_t==0:
            name1=t.text
            url=title.find('a', href=True).get('href')
            results['Name'].append(name1)
            results['URL'].append(url)

In [6]:
title_p3=soup.find('span', {'class' : 'jieshao'})#频道主题榜榜首
name4=title_p3.a.text
url4=title_p3.a['href']
results['Name'].append(name4)
results['URL'].append(url4)

In [7]:
title_p6=soup.find_all('div',{'class':'box_05_left'})[1]#作者名改为标题1
title_p6=title_p6.find_all('li')
for title6 in title_p6:
    t6=title6.find_all('a')[1]
    name6=t6.text
    url6=t6.get('href')
    results['Name'].append(name6)
    results['URL'].append(url6)

In [8]:
title_p7=soup.find_all('span',{'class':'author'}) #作者名改为标题2
for title7 in title_p7:
    parent=title7.parent.find_all('a')[1]
    name7=parent.text
    url7=parent.get('href')
    results['Name'].append(name7)
    results['URL'].append(url7)

In [9]:
results_df=pd.DataFrame.from_dict(results)  
results_df=results_df.drop(labels=range(20,25),axis=0)
results_df=results_df.drop(labels=range(118,186),axis=0)
results_df.drop_duplicates()

,Name,URL
0,先婚后爱,//www.jjwxc.net/onebook.php?novelid=5111168
1,成名[娱乐圈],//www.jjwxc.net/onebook.php?novelid=6299728
2,月亮本来没有光,//www.jjwxc.net/onebook.php?novelid=5989760
3,你听得见,//www.jjwxc.net/onebook.php?novelid=4646292
4,赤道热吻北极,//www.jjwxc.net/onebook.php?novelid=5836666
...,...,...
639,春渡,onebook.php?novelid=7087094
640,乐游原,onebook.php?novelid=7093248
641,春秋冬夏,onebook.php?novelid=7048475
642,拉住月亮,onebook.php?novelid=7061404


In [10]:
results_df.loc[results_df["URL"].str.contains('www.jjwxc.net'), "URL"]=results_df["URL"].str.slice(16,)
add_str='https://www.jjwxc.net/'
results_df["URL"]=add_str+results_df["URL"]
results_df=results_df.reset_index(drop=True)
results_df['Name'][20:30]=results_df['Name'][20:30].str.split(' ',expand=True)[0]

In [11]:
results_df

,Name,URL
0,先婚后爱,https://www.jjwxc.net/onebook.php?novelid=5111168
1,成名[娱乐圈],https://www.jjwxc.net/onebook.php?novelid=6299728
2,月亮本来没有光,https://www.jjwxc.net/onebook.php?novelid=5989760
3,你听得见,https://www.jjwxc.net/onebook.php?novelid=4646292
4,赤道热吻北极,https://www.jjwxc.net/onebook.php?novelid=5836666
...,...,...
566,春渡,https://www.jjwxc.net/onebook.php?novelid=7087094
567,乐游原,https://www.jjwxc.net/onebook.php?novelid=7093248
568,春秋冬夏,https://www.jjwxc.net/onebook.php?novelid=7048475
569,拉住月亮,https://www.jjwxc.net/onebook.php?novelid=7061404


In [12]:
novel_content={'Name':[],'Style':[], 'Finish':[], 'Words':[],'Intro':[],'Short_Intro':[],'Points':[]}

In [13]:
for u in results_df['URL']:
    novel_con=requests.get(u)
    soup_con=BeautifulSoup(novel_con.content,'lxml')
    name=soup_con.h1.text
    intro=soup_con.find('div', {'id' : 'novelintro'}).text
    intro_short=soup_con.find('span', {'style' : 'color:#F98C4D'}).text[6:]
    #keyword=soup_con.find('span', {'class' : 'bluetext'}).text[6:]
    basic=soup_con.find('ul', {'class' : 'rightul'})
    style=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.text[5:]
    finish=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text[6:]
    word=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text[6:][:-2]
    points=soup_con.find('tbody').find('span',{'itemprop':'collectedCount'}).next_sibling.next_sibling.next_sibling.strip()[5:]
    #publish=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text[5:].strip()[:-6]
    
    novel_content['Name'].append(name)
    novel_content['Intro'].append(intro)
    novel_content['Short_Intro'].append(intro_short)
    #novel_content['Keyword'].append(keyword)
    novel_content['Style'].append(style)
    novel_content['Finish'].append(finish)
    novel_content['Words'].append(word)
    novel_content['Points'].append(points)
    #novel_content['Publish'].append(publish)

In [17]:
novel_content_df=pd.DataFrame.from_dict(novel_content)  
novel_content_df

,Name,Style,Finish,Words,Intro,Short_Intro,Points
0,先婚后爱,正剧,完结\n,447910,【下本开《暗恋你的第七年》，专栏可预收】--------------------------...,只爱过你,"4,168,969,728"
1,成名[娱乐圈],正剧,完结\n,500036,徐枳追了路明一年，谈了两年不能公开的恋爱，做了三年傻子。她写的歌给路明的艺人唱，为了路明她放...,分手后爆红成顶流,"1,486,078,592"
2,月亮本来没有光,轻松,连载\n,338371,【正文完结，甜甜番外更新中~感谢支持正版~】【预收《影帝他天天被嫌弃》先婚后爱、《姐姐怎么会...,【正文完】双向暗恋的先婚后爱,"280,054,272"
3,你听得见,正剧,连载\n,179882,【每晚八点更，其余更新时间在捉虫】好友柳思嘉的生日会排场极大，在一栋灯火通明的别墅里庆祝。大...,after 17,"4,402,881,536"
4,赤道热吻北极,轻松,连载\n,337812,【正文完结，番外更新中】【下本开《月光渡我》】雨林深处，连绵的雨季让电影拍摄进度停滞不前。宋...,他终臣服于他的玫瑰。,"213,377,728"
...,...,...,...,...,...,...,...
566,春渡,正剧,连载\n,39209,疯狗从良*【一】山城的清晨，烟气笼罩，大雾四起。市井街头的拐弯处，老旧的瓦房檐下，少年一身浑...,“浮浮世事，不忘渡己”,"1,416,904"
567,乐游原,正剧,连载\n,297978,慈悲为怀，渡人渡己。善人做恶事，恶人存善心，这就是许负。当灵魂与□□背道而驰，必将死伤一方，...,慈悲为怀，渡人渡己,"1,362,206"
568,春秋冬夏,轻松,完结\n,224486,作为一个擦线上了全市最好高中的学生，叶长安发现，自己每天都卑微地在夹缝中生存，与那些大...,与老师同学们在高中的奋斗经历,"1,307,126"
569,拉住月亮,轻松,连载\n,179900,在外人面前南旌的口头禅，“滚”“死一边去”“我们只是玩玩而已，你不会当真了吧？”“你算什么东...,破镜重圆伪甜文,"1,235,948"


In [18]:
novel_content_df['Points']=novel_content_df['Points'].str.replace(',', '').astype(int)

In [19]:
novel_content_df

,Name,Style,Finish,Words,Intro,Short_Intro,Points
0,先婚后爱,正剧,完结\n,447910,【下本开《暗恋你的第七年》，专栏可预收】--------------------------...,只爱过你,4168969728
1,成名[娱乐圈],正剧,完结\n,500036,徐枳追了路明一年，谈了两年不能公开的恋爱，做了三年傻子。她写的歌给路明的艺人唱，为了路明她放...,分手后爆红成顶流,1486078592
2,月亮本来没有光,轻松,连载\n,338371,【正文完结，甜甜番外更新中~感谢支持正版~】【预收《影帝他天天被嫌弃》先婚后爱、《姐姐怎么会...,【正文完】双向暗恋的先婚后爱,280054272
3,你听得见,正剧,连载\n,179882,【每晚八点更，其余更新时间在捉虫】好友柳思嘉的生日会排场极大，在一栋灯火通明的别墅里庆祝。大...,after 17,4402881536
4,赤道热吻北极,轻松,连载\n,337812,【正文完结，番外更新中】【下本开《月光渡我》】雨林深处，连绵的雨季让电影拍摄进度停滞不前。宋...,他终臣服于他的玫瑰。,213377728
...,...,...,...,...,...,...,...
566,春渡,正剧,连载\n,39209,疯狗从良*【一】山城的清晨，烟气笼罩，大雾四起。市井街头的拐弯处，老旧的瓦房檐下，少年一身浑...,“浮浮世事，不忘渡己”,1416904
567,乐游原,正剧,连载\n,297978,慈悲为怀，渡人渡己。善人做恶事，恶人存善心，这就是许负。当灵魂与□□背道而驰，必将死伤一方，...,慈悲为怀，渡人渡己,1362206
568,春秋冬夏,轻松,完结\n,224486,作为一个擦线上了全市最好高中的学生，叶长安发现，自己每天都卑微地在夹缝中生存，与那些大...,与老师同学们在高中的奋斗经历,1307126
569,拉住月亮,轻松,连载\n,179900,在外人面前南旌的口头禅，“滚”“死一边去”“我们只是玩玩而已，你不会当真了吧？”“你算什么东...,破镜重圆伪甜文,1235948


In [20]:
novel_content_df.to_csv('jjwxc_2.csv')